In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2Model


class GPT2RewardModel(nn.Module):
    def __init__(self, model_name="gpt2"):
        super().__init__()
        self.transformer = GPT2Model.from_pretrained(model_name)
        self.value_head = nn.Linear(self.transformer.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = outputs.last_hidden_state
        value = self.value_head(last_hidden[:, -1, :])
        return value.squeeze(-1)

In [2]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

samples = [
    {
        "prompt": "What is the capital of France?",
        "chosen": "Paris is the capital of France.",
        "rejected": "France is a city in Paris.",
    },
    {
        "prompt": "What is 2 + 2?",
        "chosen": "The answer is 4.",
        "rejected": "2 plus 2 equals 22.",
    },
]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [3]:
def encode_batch(samples):
    chosen_texts = [s["prompt"] + " " + s["chosen"] for s in samples]
    rejected_texts = [s["prompt"] + " " + s["rejected"] for s in samples]

    chosen = tokenizer(chosen_texts, padding=True, truncation=True, return_tensors="pt")
    rejected = tokenizer(
        rejected_texts, padding=True, truncation=True, return_tensors="pt"
    )
    return chosen, rejected

In [ ]:
model = GPT2RewardModel()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.MarginRankingLoss(margin=1.0)

chosen, rejected = encode_batch(samples)
chosen = {k: v for k, v in chosen.items()}
rejected = {k: v for k, v in rejected.items()}

for epoch in range(200):
    model.train()
    r_chosen = model(**chosen)
    r_rejected = model(**rejected)
    target = torch.ones_like(r_chosen)

    loss = loss_fn(r_chosen, r_rejected, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.4f}")

Epoch 0 | Loss: 0.7094
